In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Hello Kagglers, Today we are goin to explore the insights of Kaggle 2021 survey data and grasp the information from visualizations.

# Import Libraries

In [ ]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)

PAL = ['#7bed7b','#ffdc52','#61caff', '#b4b4b4','#ff3b17'] # green,yellow,blue,grey,orange,
%matplotlib inline

In [ ]:
#funtion to clean plot
def setup_plot(w=1,h=1,rows=1,cols=1,X_label='',y_label='',dpi=600):
    '''
    w: width of the figsize parameter of matplotlib.pyplot
    h: height of the figsize parameter of matplotlib.pyplot
    rows: numbers of rows in gridspec
    cols: numbers of cols in gridspec
    X_label: Label of the x-axis
    y_label: Label of the y-axis
    dpi: to set dpi for the figure.dpi
    Return axes (only single axes for now, still working on the multiple axes)
    '''
    #setting plot theme
    plt.rcParams['figure.dpi'] = dpi

    fig = plt.figure(figsize=(w,h), facecolor='#f6f5f5')
    gs = fig.add_gridspec(rows, cols)
    gs.update(wspace=1.5, hspace=0.05)

    background_color = "#ffffff"
    sns.set_palette(PAL)
    
    # making single ax
    ax_dict = {}
    for row in range(rows):
        for col in range(cols):
            ax_dict["ax%s%s" %(row,col)] = fig.add_subplot(gs[row, col])

    locals().update(ax_dict)

    #setting theme for every ax in local()
    for row in range(rows):
        for col in range(cols):
                    
            for s in ["right", "top"]:
                locals()['ax' + str(row) + str(col)].spines[s].set_visible(False)
                
            locals()['ax' + str(row) + str(col)].set_facecolor(background_color)

            locals()['ax' + str(row) + str(col)].set_xlabel(X_label,fontsize=3,)
            locals()['ax' + str(row) + str(col)].set_ylabel(y_label,fontsize=3,)

            locals()['ax' + str(row) + str(col)].tick_params(labelsize=3, width=0.5, length=1.5)
            locals()['ax' + str(row) + str(col)].grid(which='major', axis='x', zorder=0, color='#EEEEEE', linewidth=0.3)
            locals()['ax' + str(row) + str(col)].grid(which='major', axis='y', zorder=0, color='#EEEEEE', linewidth=0.3)

    return locals()['ax' + str(row) + str(col)]

In [ ]:
data = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv')

In [ ]:
data.head(3)

In [ ]:
_df_ = data[data.Q3 == 'India'] # my country
print(_df_.shape)

# What is your Age?

**My age? !23**

18.1% of the people in the survery falls in my age range, 18.9% are younger and 63% people in the survery are older than me!

In [ ]:
_q = str(data.iloc[0,1])
_df = data.iloc[1:,1]
gender = data.iloc[1:,2]
_df = pd.concat([_df,gender],axis=1)

_df['sort'] = _df['Q1'].str.extract('(^[0-9]{2})', expand=False).astype(int)
_df.sort_values('sort',inplace=True, ascending=True)
_df = _df.drop('sort', axis=1)

q1 = sns.histplot(x = _df.Q1,ax=setup_plot(w=4,h=1,X_label='Age Group',y_label='Count'),zorder=4,linewidth=0.2,hue=_df.Q2,multiple='dodge')
plt.title(_q,fontsize=4, weight='bold',ha='left',loc='left')

mean=_df['Q1'].value_counts().values.mean()
median=np.median(_df['Q1'].value_counts().values)

q1.axhline(mean, color='b', linestyle='--',linewidth=0.5)
q1.axhline(median, color='g', linestyle='-',linewidth=0.5)

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

plt.text(xmin,ymax+140,'Age-group of  All Countries',fontsize=3,ha='left')
# plt.xticks(rotation=45)

q1.legend(['Mean','Median','NonBinary','Prefer to self-describe','Prefer not to say','Women','Man'], facecolor='#D8D8D8', edgecolor='#f6f5f5', fontsize=3, bbox_to_anchor=(1, 1.3), loc='upper right')

plt.show()

del gender, _df
gc.collect()


In [ ]:
_q = 'What is your age?'
gender = data.iloc[1:,1]

_x = gender.value_counts().values
_y = gender.value_counts().index

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=2,X_label='Counts',y_label='Age'),zorder=2,linewidth=0.2,color='#36ffd2',ec='black')
plt.title(_q,fontsize=4,)

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()


plt.text(xmax+1000,ymax,'Age Plot',fontsize=3,ha='right',alpha=.001) #ignore this line
for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/gender.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 100
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()

del gender, _x, _y
gc.collect()

# In which country do you currently reside?

* India :), 28.6% of the people who has given survey are from india. Followed by United States of America with 10.2% of the total number of people who gave the survey.

In [ ]:
_q = str(data.iloc[0,3])
country = data.iloc[1:,3]

_x = country.value_counts().values
_y = ['India', 'United States of America', 'Other', 'Japan', 'China',
       'Brazil', 'Russia', 'Nigeria',
       'United Kingdom', 'Pakistan',
       'Egypt', 'Germany', 'Spain', 'Indonesia', 'Turkey', 'France',
       'South Korea', 'Taiwan', 'Canada', 'Bangladesh', 'Italy', 'Mexico',
       'Viet Nam', 'Australia', 'Kenya', 'Colombia', 'Poland',
       'Iran, Islamic Republic of...', 'Ukraine', 'Singapore', 'Argentina',
       'Malaysia', 'Netherlands', 'South Africa', 'Morocco', 'Israel',
       'Thailand', 'Portugal', 'Peru', 'United Arab Emirates', 'Tunisia',
       'Philippines', 'Sri Lanka', 'Greece', 'Chile', 'Ghana', 'Saudi Arabia',
       'Ireland', 'Sweden', 'Hong Kong (S.A.R.)', 'Nepal', 'Switzerland',
       'confidential', 'Belgium', 'Czech Republic',
       'Romania', 'Austria', 'Belarus', 'Ecuador', 'Denmark', 'Uganda',
       'Kazakhstan', 'Norway', 'Algeria', 'Ethiopia', 'Iraq']

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=4,X_label='Counts',y_label='Country'),zorder=2,linewidth=0.2,color='#a578ff',ec='black')
plt.title(_q,fontsize=4)

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/country.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 200
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()

del  _x, _y
gc.collect()

# Highest level of formal education

*  Completed my bachelor's Degree in 2019

In [ ]:
_q = str(data.iloc[0,4])
plot = data.iloc[1:,4]

_x = plot.value_counts().values
_y = ['Master’s degree', 'Bachelor’s degree', 'Doctoral degree',
       'Some studies without bachelor’s degree',
       'I prefer not to answer', 'No formal education past high school',
       'Professional doctorate']

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=1,X_label='Counts',y_label='attain within 2yrs'),zorder=2,linewidth=0.2,color='#dedede',alpha=1,ec='black')
plt.title('Highest level of formal education',fontsize=4,ha='right',loc='right')

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/country.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 500
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()


# Title most similar to your current role

> *Machine Learning Engineer*

In [ ]:
_q = str(data.iloc[0,5])
plot = data.iloc[1:,5]

_x = plot.value_counts().values
_y = plot.value_counts().index

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=2,X_label='Counts',y_label='title most similar to your current role'),zorder=2,linewidth=0.2,color='#21b5ff',ec='black')
plt.title('Current Role',fontsize=4)

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

plt.text(xmax+4000,ymax,'Qualification Plot',fontsize=3,ha='right',alpha=.001) #ignore this line
for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/country.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 200
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()


# Coding Experience

I have 2+ years of experience in Coding

In [ ]:
#q3
_q = str(data.iloc[0,6])
plot = data.iloc[1:,6]

_x = plot.value_counts().values
_y = plot.value_counts().index

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=1,X_label='Counts',y_label='Coding Experience in yrs'),zorder=2,linewidth=0.2,color='#fffeb8',ec='black')
plt.title('Experience in coding Plot',fontsize=4,ha='right',loc='right')

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()


plt.text(xmax+5000,ymax,'Experience Plot',fontsize=3,ha='right',alpha=.001) #ignore this line
for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/country.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 250
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()


# Programming Language

I Love **Python**

In [ ]:
def clean(data,Question,n):
    x = []
    for i in range(1,n):
        x.append(Question+str(i))
    x.append(str('Q7_OTHER'))


    ddict = {}
    for i in x:
        ddict[data[i].value_counts().keys()[0]]=data[i].value_counts()[0]
    re = pd.DataFrame(ddict.items())
    re.iloc[:,-1] = re.iloc[:,-1].astype('int32')
    re.columns = ['Programming Languages','values']
    return re

plot = clean(data,'Q7_Part_',12)
_all = plot['values'].sum()
plot = plot.sort_values(['values'],ascending=False)

q1 = sns.barplot(y =plot['Programming Languages'],x=plot['values'],ax=setup_plot(w=1,h=1,X_label='Counts',y_label='Programming Languages'),zorder=2,linewidth=0.2,color='#6effdf',ec='black')
                   
plt.title('Most Used Programming Languages',fontsize=4, )

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

plt.text(xmax+10000,ymax,'programming languages Plot',fontsize=3,ha='right',alpha=.001) #ignore this line
for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/_all):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 600
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()

# Recommend Programming Language for Aspiring Data Scientist

True, I will also recommend python to anyone who is getting started with AI/ML etc.

In [ ]:
_q = str(data.loc[0,'Q8'])
plot = data.loc[1:,'Q8']

_x = plot.value_counts().values
_y = plot.value_counts().index

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=1,X_label='Counts',y_label='Recommend Programming Language for DS'),zorder=2,linewidth=0.2,color=PAL[2],ec='black')
plt.title('Recommend Programming Language\nfor Aspiring Data Scientist',fontsize=3,ha='right',loc='right')

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()


plt.text(xmax+7000,ymax,'Experience Plot',fontsize=3,ha='right',alpha=.001) #ignore this line
for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/country.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 550
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()


# Most Used IDE

For me, I mostly use Kaggle/Colab notebooks sometimes Spyder, I like using Jupyter Notebooks.

In [ ]:
def clean(data,Question,n):
    x = []
    for i in range(1,n):
        x.append(Question+str(i))
    x.append(str('Q9_OTHER'))


    ddict = {}
    for i in x:
        ddict[data[i].value_counts().keys()[0]]=data[i].value_counts()[0]
    re = pd.DataFrame(ddict.items())
    re.iloc[:,-1] = re.iloc[:,-1].astype('int32')
    re.columns = ['IDE\'s','values']
    return re

plot = clean(data,'Q9_Part_',12)
_all = plot['values'].sum()
plot = plot.sort_values(['values'],ascending=False).reset_index(drop=True)

plot.iloc[0,-1] = int(plot.iloc[0,-1]) + 5488
plot.drop(3,inplace=True)

q1 = sns.barplot(y =plot['IDE\'s'],x=plot['values'],ax=setup_plot(w=1,h=2,X_label='Counts',y_label='IDE\'s'),zorder=2,linewidth=0.2,color=PAL[-3],ec='black')
                   
plt.title('Most Used IDE',fontsize=4)

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

plt.text(xmax+10000,ymax,'programming languages Plot',fontsize=3,ha='right',alpha=.001) #ignore this line
for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/_all):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 600
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()


# Most Used Cloud Notebook

As I said, kaggle/Colab notebook is very easy to use, and they also provide free GPU & TPU support.

50.1% people who gave the survey uses colab notebook, followed by Kaggle 31.2% people.

In [ ]:
def clean(data,Question,n):
    x = []
    for i in range(1,n):
        x.append(Question+str(i))
    x.append(str('Q10_OTHER'))


    ddict = {}
    for i in x:
        ddict[data[i].value_counts().keys()[0]]=data[i].value_counts()[0]
    re = pd.DataFrame(ddict.items())
    re.iloc[:,-1] = re.iloc[:,-1].astype('int32')
    re.columns = ['Cloud Notebook\'s','values']
    return re

plot = clean(data,'Q10_Part_',16)
_all = plot['values'].sum()
plot = plot.sort_values(['values'],ascending=False).reset_index(drop=True)

plot.iloc[0,-1] = int(plot.iloc[0,-1]) + 5488
plot.drop(3,inplace=True)

q1 = sns.barplot(y =plot['Cloud Notebook\'s'],x=plot['values'],ax=setup_plot(w=1,h=2,X_label='Counts',y_label='Cloud Notebook\'s'),zorder=2,linewidth=0.2,color=PAL[-1],ec='black')
                   
plt.title('Most Used Cloud Notebook',fontsize=4,)

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

plt.text(xmax+10000,ymax,'programming languages Plot',fontsize=3,ha='right',alpha=.001) #ignore this line
for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/_all):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 600
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()


# Most Used Computing Platform for Data Science Project

62.5% people who gave the survey uses laptop for data science projects.

In [ ]:
_q = str(data.loc[0,'Q11'])
plot = data.loc[1:,'Q11']

_x = plot.value_counts().values
_y = ['A laptop', 'A personal computer / desktop',
       'A cloud computing platform',
       'A deep learning workstation', 'None',
       'Other']

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=1,X_label='Counts',y_label='Computing Platform'),zorder=2,linewidth=0.2,color=PAL[2],ec='black')
plt.title('Most Used Computing Platform for\nData Science Project',fontsize=3,ha='right',loc='right')

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()


plt.text(xmax+7000,ymax,'Experience Plot',fontsize=3,ha='right',alpha=.001) #ignore this line
for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/country.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 550
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()

# Current yearly compensation

My compensation is between $0-999

In [ ]:
_q = str(data.loc[0,'Q25'])
plot = data.loc[1:,'Q25']

_x = plot.value_counts().values
_y = plot.value_counts().index

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=3,X_label='Counts',y_label='current yearly compensation (approximate $USD)'),zorder=2,linewidth=0.2,color=PAL[2],ec='black')
plt.title('Current yearly compensation',fontsize=3,ha='right',loc='right')

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/country.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 550
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()

# Primary Tool Usage

I mostly prefer Google sheets and Jupyter Notebook

In [ ]:
_q = str(data.loc[0,'Q41'])
plot = data.loc[1:,'Q41']

_x = plot.value_counts().values
_y = plot.value_counts().index

q1 = sns.barplot(y =_y,x=_x ,ax=setup_plot(w=1,h=1,X_label='Counts',y_label='Tool used at work or school'),zorder=2,linewidth=0.2,color=PAL[2],ec='black')
plt.title('Primary Tool Usage',fontsize=3,ha='right',loc='right')

xmin, xmax = q1.get_xlim()
ymin, ymax = q1.get_ylim()

for p in q1.patches:
            value = f'{p.get_width():.0f} | {(p.get_width()/country.shape[0]):,.1%}'
            if value == '0':
                pass
            else:
                x = p.get_x() + p.get_width() + 550
                y = p.get_y() + p.get_height() / 2 
                q1.text(x, y, value, ha='left', va='center', fontsize=2, 
                        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round', linewidth=0.3))
plt.show()


Thank you, **WIP**.